# Pneumonia and COVID-19 Detection with Neural Networks

In [1]:
# General libraries
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns

In [14]:
!pip install imutils
!pip3 install opencv-python


     |████████████████████████████████| 52.4 MB 9.5 MB/s eta 0:00:011


In [4]:
# Machine learning libraries
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import argparse
import os

In [15]:
import cv2
from imutils import paths

In [5]:
# Load dataset paths
pneumonia_dataset = "Chest_XRay_Data"
covid_dataset = "COVID19_Data"

In [6]:
# construct the path to the COVID metadata CSV file and load it
csvPath = os.path.sep.join([covid_dataset, "metadata.csv"])
covid_meta = pd.read_csv(csvPath)

In [9]:
covid_meta.head(3)

,patientid,offset,sex,age,finding,survival,intubated,intubation_present,went_icu,in_icu,...,date,location,folder,filename,doi,url,license,clinical_notes,other_notes,Unnamed: 28
0,2,0.0,M,65.0,COVID-19,Y,NaN,NaN,NaN,NaN,...,"January 22, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
1,2,3.0,M,65.0,COVID-19,Y,NaN,NaN,NaN,NaN,...,"January 25, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-b-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
2,2,5.0,M,65.0,COVID-19,Y,NaN,NaN,NaN,NaN,...,"January 27, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-c-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN


In [21]:
# path to Pneumonia images
basePath = os.path.sep.join([pneumonia_dataset, "train", "NORMAL"])
pneu_imagePaths = list(paths.list_images(basePath))

pneu_imagePaths[:10]

['Chest_XRay_Data/train/NORMAL/IM-0115-0001.jpeg',
 'Chest_XRay_Data/train/NORMAL/IM-0117-0001.jpeg',
 'Chest_XRay_Data/train/NORMAL/IM-0119-0001.jpeg',
 'Chest_XRay_Data/train/NORMAL/IM-0122-0001.jpeg',
 'Chest_XRay_Data/train/NORMAL/IM-0125-0001.jpeg',
 'Chest_XRay_Data/train/NORMAL/IM-0127-0001.jpeg',
 'Chest_XRay_Data/train/NORMAL/IM-0128-0001.jpeg',
 'Chest_XRay_Data/train/NORMAL/IM-0129-0001.jpeg',
 'Chest_XRay_Data/train/NORMAL/IM-0131-0001.jpeg',
 'Chest_XRay_Data/train/NORMAL/IM-0133-0001.jpeg']